<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

ModuleNotFoundError: No module named 'google'

In [5]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os
import warnings
warnings.filterwarnings('ignore')

In [6]:
#DATAFILE_NAME = "reduced_dim_space_ddG.csv"
DATASET = "LARGE" # "LARGE" or "REDUCED"
if DATASET == "REDUCED":
  DATAFILE_NAME = "reduced_dim_space_ddG.csv"
  X_COLS = ['x', 'y', 'z']
else:
  DATAFILE_NAME = "large_cat_desc_col_names.csv"
  X_COLS = ['3954', '3955', '3957', '3958', '3959', '3960', '3961', '3962', '3963',
       '3964', '3965', '3966', '3967', '3968', '3969', '3970', '3971', '3972',
       '3973']

df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))

In [7]:
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)
# start with a few randomly selected catalyst
START_SAMPLE_SIZE = 3
random_catalyst_df = df.sample(START_SAMPLE_SIZE)

In [8]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=2)
    # fitting the model
    model.fit(X, y)
    return model

from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
pls_large_saved_model

PLSRegression(n_components=3)

In [9]:
original_ee = df['ddG']
def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=100, cd_iterations=10, step_size=0.001):
    if pls_model == None:
        print("Creating a new model. # data items: " + str(len(catalysts)))
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where 
            ## ee improves according to the model
            
            ## when using ddG from dataset
            #original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            original_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Found a direction for higher ddG')
    return optimized_catalysts

new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

In [28]:
random_catalyst_df[X_COLS]

,3954,3955,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973
Catalyst,,,,,,,,,,,,,,,,,,,
7_7_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
12_11_3,0.751052,0.578241,0.690619,0.648005,-0.341944,-0.750784,-0.698807,-0.726153,1.818622,1.137386,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898
6_9_5,-0.963382,0.742768,-0.317384,0.281218,2.028871,1.651922,1.575386,1.616959,-0.028326,1.054486,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633


In [27]:
new_optimized_catalysts

array([[ 2.19204621,  1.57504612,  2.19042725,  2.00550337,  0.57043889,
         0.1800573 , -0.0601493 ,  0.05597431,  0.82202323,  0.10633971,
         0.16628637,  0.39404832,  0.52829963,  0.23152003,  0.82146371,
         0.70141005,  0.76136021,  0.72248989,  0.7693672 ],
       [ 1.75105172,  1.57824082,  1.69061888,  1.6480053 ,  0.65805637,
         0.24921616,  0.30119292,  0.27384715,  2.81862175,  2.13738582,
         2.10437818,  2.78996606,  2.70797219,  0.92137107, -0.41606153,
        -0.34798358, -0.38341459,  1.49218775,  1.69189841],
       [ 0.0366176 ,  1.74276791,  0.6826161 ,  1.28121777,  3.02887051,
         2.65192225,  2.57538633,  2.616959  ,  0.97167412,  2.05448598,
         2.11273202,  1.60103942,  1.6915266 ,  2.36068392,  1.61459764,
         1.52478116,  1.57033987, -0.51845156,  0.7693672 ]])

In [13]:
# Perform k-nearest neighbors analysis
num_neighbors = 1  # Number of neighbors to consider
knn = NearestNeighbors(n_neighbors=num_neighbors)
knn.fit(df[X_COLS])  # Using all the catalysts

NearestNeighbors(n_neighbors=1)

In [32]:
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[indices[neighbor_number-1]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        if len(neighbor_catalyst) > 0: 
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,991,992,993,994,995,996,997,998,999,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
7_7_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.672194
12_11_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181818,0.181818,0.181818,...,0.0,0.0,0.181818,0.181818,0.181818,0.181818,0.272727,0.272727,0.181818,0.276786
6_9_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.463011
1_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.672194
1_1_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.276786
1_1_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.463011


In [33]:
# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
pls_predict_ee(pls_large_saved_model, properties1)

array([-0.78970154])

In [ ]:
random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [36]:
# We will now do all this in a single loop
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
prev_loop_len = len(random_catalyst_df)
while(len(random_catalyst_df) < len(df)):
    new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    if(len(random_catalyst_df) == prev_loop_len):
        print("The processing has blocked and no new catalysts are being added. Exiting.")
        break
    prev_loop_len = len(random_catalyst_df)
    print(prev_loop_len)

KeyboardInterrupt: 

In [ ]:
random_catalyst_df